In [1]:
!pip install -q duckrun --upgrade

In [2]:
import duckrun
con = duckrun.connect('bigdata/tpch.lakehouse/dbo')

🔐 Starting Azure authentication...
🖥️ Local/VS Code environment detected - trying Azure CLI first, then browser fallback
🔐 Trying Azure CLI authentication...
✅ Azure CLI authentication successful!
✅ Using existing Azure Storage token


In [3]:
try:
    con.sql("""  
        CREATE VIEW if not exists logs(file) AS SELECT 'dummy';
    SET VARIABLE list_of_files =
    (
    WITH xxxx AS (
        SELECT
        file
        FROM glob('abfss://bigdata@onelake.dfs.fabric.microsoft.com/tpch.Lakehouse/Files/DiagnosticLogs/OneLake/Workspaces/*/y=*/m=*/d=*/h=*/m=*/*.json')
        WHERE file NOT IN (SELECT DISTINCT file FROM logs) ORDER BY file
        
    )
    SELECT list(file) FROM xxxx
    );
        select *exclude(data),data.*, filename as file  from read_json_auto(GETVARIABLE('list_of_files'),hive_partitioning = true,union_by_name = 1,FILENAME = 1)
                """).write.mode("append").saveAsTable('logs')
except Exception as e:
    print(f"An error occurred: {e}")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

An error occurred: Parser Error: read_json_auto cannot take NULL list as parameter


In [4]:
con.sql(""" select originatingApp, count(*)
 from logs
  group by all
  order by 2 desc
   """).show(max_width=140)

┌──────────────────────┬──────────────┐
│    originatingApp    │ count_star() │
│       varchar        │    int64     │
├──────────────────────┼──────────────┤
│ Model                │        11306 │
│ Spark                │         5154 │
│ SqlAnalyticsEndpoint │         5078 │
│ Fabric               │         1433 │
│ object_store/0.12.4  │          795 │
│ FabricPortal         │          195 │
│ RuntimeLineage       │          114 │
│ object_store/0.12.3  │            4 │
└──────────────────────┴──────────────┘



In [5]:
con.sql(""" select 
get_workspace_name(workspaceid) as workspace_name,
get_lakehouse_name(workspaceid, itemId) as lakehouse_name,
originatingApp,operationName,httpStatusCode , 
regexp_extract(resource, '([^&/]+)/([^&/]+)/(Tables|Files)(?:/([^&/]+))?(?:/([^&/]+))?', 4) AS schema_name, count(*)
 from logs
  group by all
  order by 1,2,3 """).show(max_width=140)

🔐 Getting Fabric API token...
🖥️ Using CLI + browser fallback for Fabric API
🔐 Trying Azure CLI for Fabric API...
✅ Fabric API token obtained via Azure CLI!
✅ Using cached Fabric API token
✅ Using cached Fabric API token
✅ Using cached Fabric API token
✅ Using cached Fabric API token
✅ Using cached Fabric API token


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌────────────────┬────────────────┬──────────────────────┬─────────────────────────┬────────────────┬────────────────┬──────────────┐
│ workspace_name │ lakehouse_name │    originatingApp    │      operationName      │ httpStatusCode │  schema_name   │ count_star() │
│    varchar     │    varchar     │       varchar        │         varchar         │     int64      │    varchar     │    int64     │
├────────────────┼────────────────┼──────────────────────┼─────────────────────────┼────────────────┼────────────────┼──────────────┤
│ bigdata        │ tpch           │ Fabric               │ ReadFileOrGetBlob       │            206 │ dbo            │          184 │
│ bigdata        │ tpch           │ Fabric               │ GetFileOrBlobProperties │            404 │ dbo            │           21 │
│ bigdata        │ tpch           │ Fabric               │ ListFilePath            │            200 │                │           13 │
│ bigdata        │ tpch           │ Fabric               │ Lis

In [6]:
con.sql(""" select 
regexp_extract(resource, '([^&/]+)/([^&/]+)/(Tables|Files)(?:/([^&/]+))?(?:/([^&/]+))?', 4) AS schema_name,
get_workspace_name(workspaceid) as workspace_name,
get_lakehouse_name(workspaceid, itemId) as lakehouse_name,
originatingApp ,
 count(*) as API_calls
 from logs
  group by all
  order by API_calls desc
  limit 5
   """).show(max_width=140)

┌────────────────┬────────────────┬────────────────┬──────────────────────┬───────────┐
│  schema_name   │ workspace_name │ lakehouse_name │    originatingApp    │ API_calls │
│    varchar     │    varchar     │    varchar     │       varchar        │   int64   │
├────────────────┼────────────────┼────────────────┼──────────────────────┼───────────┤
│ aemo           │ tmp            │ data           │ Model                │     11300 │
│ CH10000        │ bigdata        │ tpch           │ SqlAnalyticsEndpoint │      2383 │
│ DiagnosticLogs │ bigdata        │ tpch           │ Spark                │      2203 │
│ Reports        │ tmp            │ data           │ Spark                │      1131 │
│ aemo           │ tmp            │ data           │ Spark                │       980 │
└────────────────┴────────────────┴────────────────┴──────────────────────┴───────────┘

